# EDA Project

### 1. Connecting to the DB

In [26]:
# Import relevant libs
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [27]:
# defines var to use
WORK_DIR = os.getcwd()
DATA_DIR = os.path.join(WORK_DIR, 'data')
CONFIG_DIR = os.path.join(WORK_DIR, 'env')

# FILES
ENV_FILE = os.path.join(CONFIG_DIR, 'db_config.env')

In [28]:
# load config from env file
load_dotenv(ENV_FILE)

True

In [29]:
db_name = os.getenv('DB_NAME')
db_name

'postgres'

#### 1.1. Connect to the DB using SQLAlchemy

In [30]:
# Load the db lib and Connect: SQAlchemy
from sqlalchemy import create_engine, text

In [31]:
# read the db credentials  and connect
db_string = os.getenv('DB_STRING')
eda_project_db = create_engine(db_string)

In [32]:
# Test with a query
query_string = """
    SET SCHEMA 'eda';
    SELECT * FROM king_county_house_details;
"""
with eda_project_db.connect() as db_conn:
    res = db_conn.execute(text(query_string)) # save the result
    print("nr. of rows" ,len(res.fetchall()))

nr. of rows 21420


#### 1.2. Connect to the DB using Psycopg2

In [12]:
# Connect to the DB with Psychopg2
import psycopg2

In [19]:
# Create connection obj
# define the connection in a method

def connect_to_psycopg2():
    conn = psycopg2.connect(
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT')
    )
    return conn.cursor()
connect_to_psycopg2()

<cursor object at 0x7f9e707e7e20; closed: 0>

In [20]:
# retrive the data
cur = connect_to_psycopg2()
cur.execute('SELECT * FROM eda.king_county_house_sales')
len(cur.fetchall())

21597

### 2. Load the db content to pandas

In [33]:
# put the query tested inside and an SQL's editor here
query_string = """
SET SCHEMA 'eda';

SELECT kchs."date", kchs.price,
kchd.bedrooms,
kchd.bathrooms,
kchd.sqft_living,
kchd.sqft_lot,
kchd.floors,
kchd.waterfront,
kchd."view",
kchd."condition",
kchd.grade,
kchd.sqft_above,
kchd.sqft_basement,
kchd.yr_built,
kchd.yr_renovated,
kchd.zipcode,
kchd.lat,
kchd."long",
kchd.sqft_living15,
kchd.sqft_lot15

FROM king_county_house_sales kchs
FULL JOIN king_county_house_details kchd
ON kchs.house_id = kchd.id;
"""

In [34]:
# Run the query directly inside pandas
# King County Housing Data (kchd)
kchd_df = pd.read_sql(query_string, eda_project_db)

In [35]:
kchd_df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2014-10-13,221900.0,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,1180.0,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0
1,2014-12-09,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,2170.0,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0
2,2015-02-25,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,770.0,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0
3,2014-12-09,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,1050.0,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0
4,2015-02-18,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,1680.0,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0


In [36]:
# Export the dataframe to a csv file
EDA_FILE_CSV = os.path.join(DATA_DIR, 'eda.csv')
kchd_df.to_csv(EDA_FILE_CSV, index=False)

In [42]:
# Now load the data from the csv file
kchd_df = pd.read_csv(EDA_FILE_CSV)
# observation (1): our data contains missing values
kchd_df.head(10)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2014-10-13,221900.0,3.0,1.00,1180.0,5650.0,1.0,NaN,0.0,3,7,1180.0,0.0,1955,0.0,98178,47.5112,-122.257,1340.0,5650.0
1,2014-12-09,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3,7,2170.0,400.0,1951,19910.0,98125,47.7210,-122.319,1690.0,7639.0
2,2015-02-25,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3,6,770.0,0.0,1933,NaN,98028,47.7379,-122.233,2720.0,8062.0
3,2014-12-09,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5,7,1050.0,910.0,1965,0.0,98136,47.5208,-122.393,1360.0,5000.0
4,2015-02-18,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3,8,1680.0,0.0,1987,0.0,98074,47.6168,-122.045,1800.0,7503.0
5,2014-05-12,1230000.0,4.0,4.50,5420.0,101930.0,1.0,0.0,0.0,3,11,3890.0,1530.0,2001,0.0,98053,47.6561,-122.005,4760.0,101930.0
6,2014-06-27,257500.0,3.0,2.25,1715.0,6819.0,2.0,0.0,0.0,3,7,1715.0,NaN,1995,0.0,98003,47.3097,-122.327,2238.0,6819.0
7,2015-01-15,291850.0,3.0,1.50,1060.0,9711.0,1.0,0.0,NaN,3,7,1060.0,0.0,1963,0.0,98198,47.4095,-122.315,1650.0,9711.0
8,2015-04-15,229500.0,3.0,1.00,1780.0,7470.0,1.0,0.0,0.0,3,7,1050.0,730.0,1960,0.0,98146,47.5123,-122.337,1780.0,8113.0
9,2015-03-12,323000.0,3.0,2.50,1890.0,6560.0,2.0,0.0,0.0,3,7,1890.0,0.0,2003,0.0,98038,47.3684,-122.031,2390.0,7570.0


### 2. Start analyzing the Data

In [43]:
# Observe the data
kchd_df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19206.000000,21534.000000,21597.000000,21597.000000,21597.000000,21145.000000,21597.000000,17749.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007602,0.233863,3.409825,7.657915,1788.596842,291.857224,1970.999676,836.650516,98077.951845,47.560093,-122.213983,1986.620318,12758.283512
std,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086858,0.765686,0.650546,1.173200,827.759761,442.490863,29.375234,4000.110554,53.513072,0.138552,0.140724,685.230472,27274.441950
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,20150.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [ ]:
# 21597 entries x 20 cols
kchd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21597 non-null  object 
 1   price          21597 non-null  float64
 2   bedrooms       21597 non-null  float64
 3   bathrooms      21597 non-null  float64
 4   sqft_living    21597 non-null  float64
 5   sqft_lot       21597 non-null  float64
 6   floors         21597 non-null  float64
 7   waterfront     19206 non-null  float64
 8   view           21534 non-null  float64
 9   condition      21597 non-null  int64  
 10  grade          21597 non-null  int64  
 11  sqft_above     21597 non-null  float64
 12  sqft_basement  21145 non-null  float64
 13  yr_built       21597 non-null  int64  
 14  yr_renovated   17749 non-null  float64
 15  zipcode        21597 non-null  int64  
 16  lat            21597 non-null  float64
 17  long           21597 non-null  float64
 18  sqft_l